In [142]:
import pandas as pd
import numpy as np
input_file = "input.txt"
with open(input_file) as file:
    memory_map = file.read().strip()
memory_map



'692094513253604282899448234539616972499153261626907217394161512944107098953354935354419233821564748010756124583228586113546912893865623036233381811143374634475189412580544832167441833745944043647640252326425751262138384939496885473972164470523126636153447614102264683866384498168566572893662415123547679158492982248229298733447982401688819695702442567393828975369862124415713097598939622673154945429814439168646856282037639023513510833023859031667426144078303958159458464624762971998658256020359824431259698527983298452680855940591588106738359350238410749458672497692569494821663595247570344596385427125917564927157637818460307494659827644524864084186047993336852499692594527474928226406314892241639963248537609748695266378764363331635694946979813415395034145591773734911554839053553653639723765451955667971618565259557275474967286634555391157662816161433886188065878163738283789868688946324912611644717628512830356013415443109961277216228979713130364165501551677169604695922858524129964658137053808

In [ ]:
from collections import namedtuple
Block = namedtuple('Block', ['id', 'size'])

blocks = []
for index, character in enumerate(list(memory_map)):
    id = int(index / 2) if index % 2 == 0 else None
    blocks.append(Block(id=id, size=int(character)))
blocks = pd.Series(blocks)
    

0           (0, 6)
1        (None, 9)
2           (1, 2)
3        (None, 0)
4           (2, 9)
           ...    
19994    (9997, 1)
19995    (None, 3)
19996    (9998, 9)
19997    (None, 2)
19998    (9999, 7)
Length: 19999, dtype: object

In [ ]:
def has_holes(disk):
        first_hole_index = disk[disk.apply(lambda x: pd.isnull(x.id))].index.min()
        last_block_index = disk[disk.apply(lambda x: pd.notnull(x.id))].index.max()
        return first_hole_index < last_block_index
    
def shift_blocks(disk):
    temp_disk = disk
    while has_holes(temp_disk):
        first_hole_index = temp_disk[temp_disk.apply(lambda x: pd.isnull(x.id))].index.min()
        last_block_index = temp_disk[temp_disk.apply(lambda x: pd.notnull(x.id))].index.max()

        hole = temp_disk[first_hole_index]
        last_block = temp_disk[last_block_index]
        
        elements_to_shift = min(hole.size, last_block.size)
        
        # get new blocks for hole
        hole_replacement = [Block(id=last_block.id, size=elements_to_shift)]
        if elements_to_shift < hole.size:
            hole_size_left = hole.size - elements_to_shift
            hole_replacement.append(Block(id=None, size=hole_size_left))
        
        # last block replacement
        last_block_replacement = []
        
        if elements_to_shift < last_block.size:
            last_block_replacement.append(Block(id=last_block.id, size=last_block.size-elements_to_shift))
        last_block_replacement.append(Block(id= None, size=elements_to_shift))
        

        temp_disk = pd.Series(list(temp_disk[:last_block_index]) + last_block_replacement + list(temp_disk[last_block_index+1:]))
        temp_disk = pd.Series(list(temp_disk[:first_hole_index]) + hole_replacement + list(temp_disk[first_hole_index+1:]))
        
    return temp_disk


shifted_disk = shift_blocks(blocks)
def calculate_points(disk):
    index = 0
    sum = 0
    for block in disk:
        if block.id == None:
            index += block.size
            continue
        for spot in range(0, block.size):
            index_of_spot = index + spot
            sum += (index_of_spot*block.id)
        index += block.size
    return sum
        
calculate_points(shifted_disk)

shifted


# Part 2

In [147]:
def move_files_as_a_whole(disk):
    temp_disk = list(disk)
    max_file_id = disk[disk[disk.apply(lambda x: pd.notnull(x.id))].index.max()].id
    
    for file_id in reversed(range(0, max_file_id+1)):
        index_of_file = next((i for i in reversed(range(0, len(temp_disk))) if temp_disk[i].id == file_id), None)
        file = temp_disk[index_of_file]
        index_of_free_hole = next((i for i in range(0, index_of_file) if temp_disk[i].size >= file.size and temp_disk[i].id == None), None)   
        if index_of_free_hole == None:
            continue
        hole = temp_disk[index_of_free_hole]
        temp_disk[index_of_file] = Block(id=None, size=file.size)
        temp_disk
        del temp_disk[index_of_free_hole]
        temp_disk.insert(index_of_free_hole, file)
        if hole.size > file.size:
            temp_disk.insert(index_of_free_hole+1, Block(id=None, size=hole.size-file.size))
    return temp_disk

print(blocks)
shifted_disk = move_files_as_a_whole(disk=blocks)
shifted_disk


0           (0, 6)
1        (None, 9)
2           (1, 2)
3        (None, 0)
4           (2, 9)
           ...    
19994    (9997, 1)
19995    (None, 3)
19996    (9998, 9)
19997    (None, 2)
19998    (9999, 7)
Length: 19999, dtype: object


[Block(id=0, size=6),
 Block(id=9999, size=7),
 Block(id=9997, size=1),
 Block(id=9982, size=1),
 Block(id=1, size=2),
 Block(id=None, size=0),
 Block(id=2, size=9),
 Block(id=9996, size=2),
 Block(id=9992, size=2),
 Block(id=3, size=5),
 Block(id=9980, size=1),
 Block(id=4, size=3),
 Block(id=9981, size=2),
 Block(id=5, size=5),
 Block(id=9990, size=3),
 Block(id=6, size=6),
 Block(id=None, size=0),
 Block(id=7, size=4),
 Block(id=9974, size=1),
 Block(id=9961, size=1),
 Block(id=8, size=8),
 Block(id=9952, size=2),
 Block(id=9, size=8),
 Block(id=9998, size=9),
 Block(id=10, size=9),
 Block(id=9994, size=4),
 Block(id=11, size=4),
 Block(id=9995, size=8),
 Block(id=12, size=2),
 Block(id=9976, size=3),
 Block(id=13, size=4),
 Block(id=9988, size=4),
 Block(id=9949, size=1),
 Block(id=14, size=3),
 Block(id=9993, size=7),
 Block(id=9950, size=2),
 Block(id=15, size=6),
 Block(id=9948, size=1),
 Block(id=16, size=6),
 Block(id=9991, size=6),
 Block(id=9975, size=3),
 Block(id=17, size=

In [148]:
calculate_points(shifted_disk)

6353648390778